### Pneumatic cylinder - mechanical subsystem
The pneumatic cylinder is modelled a mass subjected to two pressures, viscous dissipation and Coulomb friction.  
$$ (m_p + m_L)\ddot{x} + c_p \dot{x} = F_L + p_1 A_1 -p_2 A_2 -p_A A_r -F_f(\dot{x})$$  
Where $m_p$ and $m_L$ are the masses of the piston-rod and load, $c_p$ is the viscous damping coefficient, $F_L$ is the load force, $p_1, A_1$ is the pressure and area on the retraction side, $p_2, A_2$ is for the extension side, $p_A, A_r$ is the surrounding pressure and rod area and $F_f$ is the Coulomb friction given by  
$$
F_f = \left\{ \begin{array}{cl}
F_{sf} & \text{if } \dot{x} = 0 \\
F_{df}\operatorname{sgn}(\dot{x}) & \text{if } \dot{x} \neq 0
\end{array}\right.
$$

In first-order form the system can be written as
$$
\dot{x}_1 = x_2 \\
\dot{x}_2 = \frac{1}{m_p+m_L}\left( F_L + p_1 A_1 -p_2 A_2 -p_A A_r -F_f(\dot{x}_2) - c_p \dot{x}_2 \right)
$$  
It can be seen that the mechanical subsystem is coupled to the pneumatic subsystem through the pressure states $p_1$ and $p_2$.
### Pneumatic cylinder - pneumatic subsystem


The volume of air is modelled by using the ideal gas equation, continuity (mass conservation) and the energy equation.  
$$ p = \rho R T \\
\dot{m} = \frac{\text{d}}{\text{d}t} \left( \rho V \right ) \\
q_{in} - q_{out} + k C_v \left( \dot{m}_{in} T_{in} - \dot{m}_{out} T_{out} \right)- \dot{W} = \dot{U} 
$$
The rate of change of work is rate of change of volume at constant pressure
$$ \dot{W} = p\dot{V} $$  

Combining these equations, using the ideal gas relation $C_v = R/(k-1)$ and assuming $T_{in}=T_{out}$ we get the equation
$$
\frac{k-1}{k}(q_{in} - q_{out}) + \frac{1}{\rho}(\dot{m}_{in} - \dot{m}_{out}) -\dot{V} =  \frac{V}{k p} \dot{p}
$$

We can assume an adiabatic process ($q_{in}-q_{out}=0$) or isothermal ($T=\text{constant}$) and get
$$
\text{Adiabatic: } \dot{p} = k\frac{R T}{V}(\dot{m}_{in} - \dot{m}_{out}) - k\frac{p}{V}\dot{V} \\
\text{Isothermal: } \dot{p} = \frac{R T}{V}(\dot{m}_{in} - \dot{m}_{out}) - \frac{p}{V}\dot{V} \\
$$
As the only difference is the specific heat ratio $k$ we can make a model such as
$$
\dot{p} = \frac{R T}{V}(\alpha_{in}\dot{m}_{in} - \alpha_{out}\dot{m}_{out}) - \alpha \frac{p}{V}\dot{V} \\
$$
where the three coefficients $\alpha \in [1, k]$ represent the heat transfer characteristics.  
For charging: $\alpha_{in}=k$. For discharging $\alpha_{out}= 1 $. The compression/expansion due to the piston motion is best described by $\alpha = 1.2$

In [1]:
class plant(object):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

### Connecting tubes
The dynamics of the flow in the connecting tubes can be described by a dispersive hyperbolic PDE.  
Neglecting the higher order dynamics the equation reduces to a 1D wave equation. The effect of the tubes are therefore a time delay $\tau = L_t/c$ for the pressure to reach the end, and an attenatuation of the pressure due to friction $\phi = e^{-R_t \tau \frac{R T}{2 p}}$  
where $p$ is the end pressure. The tube resistance $R_t$ can be estimated for a laminar or fully turbulent flow as
$$
R_t = \left\{\begin{array}{cl} 
\frac{32\mu}{D^2} & \text{Laminar} \\
0.158 \frac{\mu}{D^2} \operatorname{Re}^{3/4}  & \text{Turbulent}
\end{array}\right.
$$ 
With $h(t)$ begin the mass flow at the inlet of the tube, the mass flow at the outlet is
$$
\dot{m}_t(L_t,t) = \left\{\begin{array}{cl} 
0 & \text{if } t < \tau \\
\phi(R_t) \, h \left( t-\frac{L_t}{c}\right) & \text{if } t > \tau
\end{array}\right.
$$ 

### Valve model
This model has a spring at both sides so the rest position is in the middle.
$$
M_s\ddot{x}_s + c_s \dot{x}_s +F_f +2 k_s x_s = F_c
$$
In the case of current control: $F_c = K_{fc} i_c$.

The Festo MPYE valve is voltage controlled between 0-10 V. It may have have a resting position at the side, so it requires a bias voltage. It can probably be modelled as
$$
M_s\ddot{x}_v = -c_s \dot{x}_s -k_s(x_s +x_{s0}) + F_c -F_f
$$
The equilibrium is $F_{c} = k_s (x_{v0} + x_{s0}) $  

$$
M_v\ddot{x}_v = -c_v \dot{x}_v -k_s(x_v -x_{s0}) + F_v -F_f
$$


### Valve orifice area
The valve position $x_v$ (relative to middle) controls the orifice area. In turn, the orifice area and pressures govern the flow rate through the valve. A model is
$$
\dot{m}_v = \left\{\begin{array}{cl}
C_f A_v C_1\frac{P_u}{\sqrt{T}} & \text{if } \frac{P_d}{P_u} \leq P_{cr}\\
C_f A_v C_2\frac{P_u}{\sqrt{T}} \left(\frac{P_d}{P_u} \right)^{1/k} \sqrt{1 -\left(\frac{P_d}{P_u} \right)^{(k-1)/k} } & \text{if } \frac{P_d}{P_u} >P_{cr}
\end{array}\right.
$$

The variables in this equation are the valve orifice area $A_v$ and one of the pressures (supply pressure is about constant).  
$C_1$, $C_2$ only depend on $R$ and $k$, and $P_{cr}$ only on $k$. $P_{cr}$ indicates when the flow is choked or unchoked.

In [5]:
import numpy as np
Rs = 287.058 # J/kg.K specific gas constant
k = 1.4
C1 = np.sqrt(k/Rs *(2/(k+1))**((k+1)/(k-1))); print('C1 = ',C1)
C2 = np.sqrt(2*k/Rs/(k-1)); print('C2 = ',C2)
Pcr = (2/(k+1))**(k/(k-1)); print('Pcr = ',Pcr)

C1 =  0.04041433642177708
C2 =  0.1561579836560165
Pcr =  0.5282817877171742
